## Training a character language model and studying various ways of generating text

**Author: matthieu.labeau@telecom-paris.fr**

## Objectives:

- We will train a network to predict a next character given an input sequence of characters, and use it to generate new sequences.
- We will strictly work with local (and not structured, meaning we will only one character at a time) prediction - however, we will look into a relatively simple heuristic to improve the "structure": *beam search*. We can also try to improve generation with other methods: *temperature* sampling, *top-k* sampling, *top-p* sampling,
- We will use ```keras```to build the model based on a **recurrent neural network** called a **LSTM**, which will use simple features (one-hot vector representing previous characters) to predict the next characters. We will use a small model to avoid training for too long. *Remark: you don't need to know how this model works - just its inputs and outputs !*
- We will use a small dataset (poetry, from project Gutenberg) - you can use any data you prefer, as long as you are able to train the model on it.
- Even with a small dataset and a small model, training may be long. If you can use a computing infrastructure, like Google colab, it may be more practical - and you probably can obtain better results by using a bigger model and a larger dataset.

#### Obtaining the data
- We download directly the ebook from project Gutenberg - you can get any other text you would prefer.

In [ ]:
!pip install keras tensorflow

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.20.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (620.4 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl (30 kB)
  Using cached wrapt-1.17.3-cp310-cp310-manylinux1_x86_64.manylinux_2_28_x86_64.manylinux_2_5_x86_64.whl (81 kB)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl (24.5 MB)
  Using cached termcolor-3.1.0-py3-none-any.whl (7.7 kB)
  Using cached tensorboard-2.20.0-py3-none-any.whl (5.5 MB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl (71 kB)
  Using cached gast-0.6.0-py3-none-any.whl (21 kB)
  Using cached grpcio-1.75.0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (6.5 MB)
  Using cached protobuf-6.32.1-cp39-abi3-manylinux2014_x86_64.whl (322 kB)
  Using cached tensorboard_data_server-0.7.2-p

In [71]:
from keras.utils import get_file
url = 'http://www.gutenberg.org/cache/epub/6099/pg6099.txt'
path = get_file('pg6099.txt', origin=url)

f = open(path, 'r' , encoding = 'utf8')
lines = f.readlines()
text = []

start = False
for line in lines:
    if("*** START OF THE PROJECT GUTENBERG EBOOK LES FLEURS DU MAL ***" in line and start==False):
        start = True
    if("            *** END OF THE PROJECT GUTENBERG EBOOK LES FLEURS DU MAL ***" in line):
        break
    if(start==False or len(line) == 0):
        continue
    text.append(line)

f.close()
text = " ".join(text)

In [72]:
print(text[20000:21000])

ces plaintes,
   Ces extases, ces cris, ces pleurs, ces _Te Deum,_
   Sont un écho redit par mille labyrinthes;
   C'est pour les coeurs mortels un divin opium.
 
   C'est un cri répété par mille sentinelles,
   Un ordre renvoyé par mille porte-voix;
   C'est un phare allumé sur mille citadelles,
   Un appel de chasseurs perdus dans les grands bois!
 
   Car c'est vraiment, Seigneur, le meilleur témoignage
   Que nous puissions donner de notre dignité
   Que cet ardent sanglot qui roule d'âge en âge
   Et vient mourir au bord de votre éternité!
 
 
 
 
   LA MUSE VENALE
 
 
   O Muse de mon coeur, amante des palais,
   Auras-tu, quand Janvier lâchera ses Borées,
   Durant les noirs ennuis des neigeuses soirées,
   Un tison pour chauffer tes deux pieds violets?
 
   Ranimeras-tu donc tes épaules marbrées
   Aux nocturnes rayons qui percent les volets?
   Sentant ta bourse à sec autant que ton palais,
   Récolteras-tu l'or des voûtes azurées?
 
   Il te faut, pour gagner ton pain de chaq

#### Keeping track of possible characters
- Using a ```set```, create a sorted list of possible characters
- Create two dictionnaries, having characters and corresponding indexes as {key: value}, and reverse.

Example:

```python
chars = [a, b, c]
```

```python
chars_indices = {a: 0, b: 1, c: 2}
```

```python
indices_chars = {0: a, 1: b, 2: c}
```

In [73]:
print('Corpus length:', len(text))  

chars = sorted(list(set(text)))
print(f"Unique char list:{chars[::10]}")
print('Total number of characters:', len(chars))
char_indices = {char: idx for idx, char in enumerate(chars)}
indices_char = {idx: char for idx, char in enumerate(chars)}
print(char_indices)
print(indices_char)

Corpus length: 179332
Unique char list:['\n', '-', '7', 'E', 'O', 'Y', 'g', 'q', '«', 'ê', '’']
Total number of characters: 105
{'\n': 0, ' ': 1, '!': 2, '$': 3, '%': 4, "'": 5, '(': 6, ')': 7, '*': 8, ',': 9, '-': 10, '.': 11, '/': 12, '0': 13, '1': 14, '2': 15, '3': 16, '4': 17, '5': 18, '6': 19, '7': 20, '8': 21, '9': 22, ':': 23, ';': 24, '?': 25, 'A': 26, 'B': 27, 'C': 28, 'D': 29, 'E': 30, 'F': 31, 'G': 32, 'H': 33, 'I': 34, 'J': 35, 'K': 36, 'L': 37, 'M': 38, 'N': 39, 'O': 40, 'P': 41, 'Q': 42, 'R': 43, 'S': 44, 'T': 45, 'U': 46, 'V': 47, 'W': 48, 'X': 49, 'Y': 50, '[': 51, ']': 52, '_': 53, 'a': 54, 'b': 55, 'c': 56, 'd': 57, 'e': 58, 'f': 59, 'g': 60, 'h': 61, 'i': 62, 'j': 63, 'k': 64, 'l': 65, 'm': 66, 'n': 67, 'o': 68, 'p': 69, 'q': 70, 'r': 71, 's': 72, 't': 73, 'u': 74, 'v': 75, 'w': 76, 'x': 77, 'y': 78, 'z': 79, '«': 80, '»': 81, 'È': 82, 'É': 83, 'Ï': 84, 'à': 85, 'â': 86, 'ç': 87, 'è': 88, 'é': 89, 'ê': 90, 'ë': 91, 'î': 92, 'ï': 93, 'ô': 94, 'ù': 95, 'û': 96, 'ü': 97

#### Creating training data
- We will represent characters using *one-hot vectors*. Hence, the i-th character of n possible characters will be represented by a vector of length $n$, containing $0$ expect for a $1$ in position $i$. Following our previous examples, ```a = [1, 0, 0]``` and ```b = [0, 1, 0]```.
- Hence, a sequence of characters is a list of one-hot vectors. Our goal will be to predict, given an input sequence of fixed length (here, this length is given by ```maximum_seq_length```) the next character. Hence, we need to build two lists: ```sentences```, containing the input sequences, and ```next_char``` the characters to be predicted.
- We do not necessarily need to take all possible sequences. We can select one every ```time_step``` steps.

Example: Using the previous dictionnaries, the sequence:
```'acabbaccaabba'``` with ```maximum_seq_length = 4``` and ```time_step = 2``` would give the following lists:

```python
sentences = ['acab', 'abba', 'bacc', 'ccaa', 'aabb']
```

```python
next_char = ['b', 'c', 'a', 'b', 'a']
```

In [74]:
maximum_seq_length = 24
time_step = 1
sentences = []
next_char = []
for i in range(0, len(text)- maximum_seq_length, time_step):
    sentences.append(text[i:i+maximum_seq_length])
    next_char.append(text[i+maximum_seq_length])

print('Number of Sequences:', len(sentences))

Number of Sequences: 179308


In [75]:
import numpy as np
import random
import sys

#### Creating training tensors
- We need to transform these lists into tensors, using one-hot vectors to represent characters.
- We will need 3 dimensions for the training examples from ```sentences```: the number of examples, the length of the sequence, and the dimension of the one-hot vector
- This is reduced to 2 dimensions for the ```next_char```: number of examples and one-hot vector.

Example: the previous ```sentences``` would become:

```python
X = [[[1, 0, 0],
      [0, 0, 1],
      [1, 0, 0],
      [0, 1, 0]],
     [[1, 0, 0],
      [0, 1, 0],
      [0, 1, 0],
      [1, 0, 0]],
     [[0, 1, 0],
      [1, 0, 0],
      [0, 0, 1],
      [0, 0, 1]],
     [[0, 0, 1],
      [0, 0, 1],
      [1, 0, 0],
      [1, 0, 0]],
     [[1, 0, 0],
      [1, 0, 0],
      [0, 1, 0],
      [0, 1, 0]]]
```
       
```python
y = [[0, 1, 0],
     [0, 0, 1],
     [1, 0, 0],
     [0, 1, 0],
     [1, 0, 0]]
```

In [76]:
num_chars = len(chars)
X = np.zeros((len(sentences), maximum_seq_length, num_chars), dtype=np.bool_)
y = np.zeros((len(sentences), num_chars), dtype=np.bool_)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_char[i]]] = 1

#### Implement the model
In order to implement the model as simply as possible, we will use ```keras```. It allows to create models with only a few lines of code.
First, we will create a very simple model based on a **LSTM**, which is a *recurrent* architecture. Note that one the strength of a recurrent architecture is to allow for inputs of varying length - here, to simplify data processing, we will keep a **fixed input size**.

In [77]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.callbacks import LambdaCallback, EarlyStopping

We need to create a LSTM model that takes directly out inputs from ```X``` and try to predict one-hot vectors from ```y```.
- What are the input and output dimensions ?
  - ```X```: size of the dataset $\times$ maximum sequence length $\times$ vocabulary size
  - ```y```: size of the dataset $\times$ vocabulary size
- The model should be made with a ```LSTM``` layer, and a ```Dense``` layer followed by a softmax activation function. Work out the intermediate dimensions:
  - ```X``` $\rightarrow$ (LSTM) $\rightarrow$ ```h``` $\rightarrow$ (Dense) $\rightarrow$ ```s``` $\rightarrow$ (softmax) $\rightarrow$ ```pred```
  - Look at layers arguments and find out to proper ```input_shape``` for the ```LSTM``` layer and the proper size for the ```Dense``` layer.
  - We can use 256 as the size of hidden states for the ```LSTM```.
- We will minimize ```cross-entropy(pred, y)```. Use the ```categorical_crossentropy``` loss, with the optimizer of your preference (for example, ```adam```).

In [29]:
model = Sequential()
model.add(LSTM(256, input_shape=(maximum_seq_length, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

2025-09-22 11:00:35.763913: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/ambroise012/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


We will now only need a few functions to use this model:
- ```model.fit```, which you will call on the appropriately processed data ```X, y```
- ```model.predict```, which we will use on an input **of the same dimension of X** to output the probabilities. That includes the *first one*, corresponding to the number of examples in the input.

#### Create a function to generate text with our model
- We use the output of our model to select the next most probable character (with the ```argmax``` function)
- We need to transform an input text into an input tensor, as before (taking the right length, the last ```maximum_seq_length``` characters)
- We need to transform back the most probable index into a character and add it to our text.
- This must be looped ```num_generated``` times, each time obtaining a new input tensor from the new input sequence (which has the character we previously predicted at the end !)


We can begin by writing a function facilitating the transfer between text and tensors:

In [ ]:
def get_tensor(sentence, voc):
    x = np.zeros((1, len(sentence), len(voc)))
    for t, char in enumerate(sentence):
        x[0, t, voc[char]] = 1.
    return x

The following function (```end_epoch_generate```) is here to facilitate automatic generation at the end of each epoch, so you can monitor of generation changes as the model trains. It calls the ```generate_next``` function upon each sequence of text in ```texts_ex```. The only element in this list right now comes from the training data - you can add your own.

In [81]:
def generate_next(model, text, num_generated=120):
    """ 
    Generates new text, character by character, using a trained model. 
    It starts from an initial input text and predicts the next character repeatedly for a specified number of steps.
    """
    generated = text
    sentence = text[-maximum_seq_length:]
    for i in range(num_generated):
        x = get_tensor(sentence, char_indices)
        predictions = model.predict(x,verbose=0)[0]
        next_index = np.argmax(predictions)
        next_char = indices_char[next_index] 
        generated += next_char
        sentence = sentence[1:] + next_char
    return(generated)

def end_epoch_generate(epoch, _):
    print('\n Generating text after epoch: %d' % (epoch+1))
    texts_ex = ["La sottise, l'erreur, le péché"]
    for text in texts_ex:
        sample = generate_next(model, text)
        print('%s' % (sample))

In [68]:
text_ex = "La sottise, l'erreur, le péché"
generate_next(model, text_ex)

"La sottise, l'erreur, le péchéù•0ùtB0Bkqk[t[.K?»çQn‘Éùù   ù  ù  ù  ù  ù  ù  ù  ù  ù  ù  ù  ù  ù  ù  ù  ù  ù  ù  ù  ù  ù  ù  ù  ù  ù  ù  ù  ù  ù  ù  ù "

In [69]:
model.fit(X, y,
          batch_size=128,
          epochs=10,
          validation_split = 0.2,
          callbacks=[LambdaCallback(on_epoch_end=end_epoch_generate)])

Epoch 1/10
1120/1121 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 2.8860
 Generating text after epoch: 1
La sottise, l'erreur, le péchése de coure de coure de coure de coure de coure de coure de coure de coure de coure de coure de coure de coure de coure 
1121/1121 ━━━━━━━━━━━━━━━━━━━━ 94s 82ms/step - loss: 2.5372 - val_loss: 2.7942
Epoch 2/10
1120/1121 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 2.1032
 Generating text after epoch: 2
La sottise, l'erreur, le péchéses de soures et le soures et le soures et le soures et le soures et le soures et le soures et le soures et le soures et
1121/1121 ━━━━━━━━━━━━━━━━━━━━ 87s 78ms/step - loss: 2.0646 - val_loss: 2.7545
Epoch 3/10
1121/1121 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 1.9656
 Generating text after epoch: 3
La sottise, l'erreur, le péchés des les ples les ples les ples les ples les ples les ples les ples les ples les ples les ples les ples les ples les pl
1121/1121 ━━━━━━━━━━━━━━━━━━━━ 86s 77ms/step - loss: 1.9415 - val_loss: 2.6912
Epoc

#### Using character embeddings
- Instead of using one-hot vectors to represent characters, we will now use character embeddings, which are vectors belonging to the same space.
- We will need as many vectors as there is characters. The input of the network will be simpler, since we will just need to indicate to the model which character is in input.
- The output does not change: indeed, Keras uses one-hot vectors for the target of the categorical cross-entropy loss.
Example: the previous example ```sentences``` would now become:
- We need to add a ```Embedding``` layer to the model, with the right input size, and to choose which dimension use for our embeddings.

```python
X = [[0, 2, 0, 1],
     [0, 1, 1, 0],
     [1, 0, 2, 2],
     [2, 2, 0, 0],
     [0, 0, 1, 1]]
```
       
```python
y = [[0, 1, 0],
     [0, 0, 1],
     [1, 0, 0],
     [0, 1, 0],
     [1, 0, 0]]
```

In [79]:
X_emb = np.zeros((len(sentences), maximum_seq_length), dtype=int)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X_emb[i,t] = char_indices[char]

In [80]:
from keras.layers import Embedding

model_emb = Sequential()
model_emb.add(Embedding(len(chars), 32, input_length = maximum_seq_length))
model_emb.add(LSTM(256))
model_emb.add(Dense(len(chars)))
model_emb.add(Activation('softmax'))

model_emb.compile(loss='categorical_crossentropy', optimizer='adam')

/home/ambroise012/.local/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [82]:
model_emb.fit(X_emb, y,
          batch_size=128,
          epochs=10,
          validation_split = 0.2,
          callbacks=[LambdaCallback(on_epoch_end=end_epoch_generate)])

Epoch 1/10
1121/1121 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 2.7910
 Generating text after epoch: 1
La sottise, l'erreur, le péché de la pare de la pare de la pare de la pare de la pare de la pare de la pare de la pare de la pare de la pare de la par
1121/1121 ━━━━━━━━━━━━━━━━━━━━ 103s 88ms/step - loss: 2.4251 - val_loss: 2.7580
Epoch 2/10
1121/1121 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 2.0230
 Generating text after epoch: 2
La sottise, l'erreur, le péché de la pare de la pare de la pare de la pare de la pare de la pare de la pare de la pare de la pare de la pare de la par
1121/1121 ━━━━━━━━━━━━━━━━━━━━ 92s 82ms/step - loss: 1.9798 - val_loss: 2.6977
Epoch 3/10
1121/1121 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 1.8653
 Generating text after epoch: 3
La sottise, l'erreur, le péché de la pare de la pare de la pare de la pare de la pare de la pare de la pare de la pare de la pare de la pare de la par
1121/1121 ━━━━━━━━━━━━━━━━━━━━ 92s 82ms/step - loss: 1.8393 - val_loss: 2.6555
Epo

#### Sampling with our model
- Now, instead of simply selecting the most probable next character, we would like to be able to draw a sample from the distribution output by the model.
- To better control the generation, we would like to use the argument ```temperature```, to smooth the distribution.
- We will use the ```multinomial``` function from the ```random``` package to draw samples.
- We integrate this into a function ```generate_sample``` that is almost exactly like ```generate_next```.

### 🔹 1. Température appliquée à une distribution

Ton modèle sort une distribution de probabilités $p_i$ (ex. via `softmax`).
On veut la transformer avec la **température $T$** :

$$
q_i = \frac{\exp\left(\frac{\log(p_i)}{T}\right)}{\sum_j \exp\left(\frac{\log(p_j)}{T}\right)}
$$

👉 en pratique c’est équivalent à :

$$
q_i = \frac{p_i^{1/T}}{\sum_j p_j^{1/T}}
$$

* Si $T=1$, alors $q_i = p_i$ (pas de changement).
* Si $T < 1$, les grandes probabilités deviennent encore plus grandes → choix plus déterministe.
* Si $T > 1$, les probabilités s’aplatissent → plus de hasard.

---

### 🔹 2. Tirage multinomial (`sample`)

Une fois que tu as $q_i$, tu tires **un index** aléatoirement selon cette distribution :

$$
\text{next\_index} \sim \text{Multinomial}(1, q)
$$

C’est-à-dire :

* Tu tires un seul échantillon.
* La probabilité de tirer l’index $i$ est exactement $q_i$.

---

### 🔹 3. Génération de texte

Le processus itératif est :

$$
\text{seq}_{t+1} = \text{seq}_t \;+\; \text{next\_char}
$$

avec

$$
\text{next\_char} = \text{argmax}(\text{sample}(q))
$$

ou plus simplement "le caractère choisi aléatoirement selon la distribution tempérée".

---

✅ Donc dans ton code :

* **`reweight(predictions, temperature)`** applique la formule

  $$
  q_i = \frac{p_i^{1/T}}{\sum_j p_j^{1/T}}
  $$

* **`sample(predictions, temperature)`** tire un indice selon $q$.



In [85]:
def reweight(predictions, temperature):
    predictions = np.asarray(predictions).astype('float64')
    log_predictions = np.log(predictions) / temperature
    predictions = np.exp(log_predictions)
    predictions = predictions / np.sum(predictions)
    return predictions

def sample(predictions, temperature):
    predictions = reweight(predictions, temperature)
    sampled = np.random.multinomial(1, predictions, 1)
    return np.argmax(sampled)

def generate_sample(model, text, num_generated=120, temperature=1.0):
    generated = text
    sentence = text[-maximum_seq_length:]
    for i in range(num_generated):
        x = np.zeros((1, maximum_seq_length))
        for t, char in enumerate(sentence):
            x[0, t] = char_indices[char]
        predictions = model.predict(x, verbose=0)[0]
        next_index = sample(predictions, temperature)
        next_char = indices_char[next_index]
        generated += next_char
        sentence = sentence[1:] + next_char
    return(generated)

In [86]:
print(generate_sample(model_emb, text_ex, temperature = 0.7))

La sottise, l'erreur, le péchés que dont de sourir;
 
   La Morts de la Plaisir comme un soleil opard, charché ma forte,
   Sous dont le sous le nouve


In [87]:
def sample_top_k(predictions, temperature, k):
    predictions = np.asarray(predictions).astype('float64')
    log_predictions = np.log(predictions) / temperature
    indices_to_remove = log_predictions.argsort()[:k]
    log_predictions[indices_to_remove] = -float('Inf')
    predictions = np.exp(log_predictions)
    predictions = predictions / np.sum(predictions)
    probas = np.random.multinomial(1, predictions, 1)
    return np.argmax(probas)


def generate_sample_top_k(model, text, num_generated=120, temperature=1.0, k=10):
    generated = text
    sentence = text[-maximum_seq_length:]
    for i in range(num_generated):
        x = np.zeros((1, maximum_seq_length))
        for t, char in enumerate(sentence):
            x[0, t] = char_indices[char]
        predictions = model.predict(x, verbose=0)[0]
        next_index = sample_top_k(predictions, temperature, k)
        next_char = indices_char[next_index]
        generated += next_char
        sentence = sentence[1:] + next_char
    return(generated)

In [88]:
print(generate_sample_top_k(model_emb, text_ex, temperature = 0.8, k = 3))

La sottise, l'erreur, le péchés de son oeil ficle bier qu'un sombre comme un gâte avec la longue
   Appernelle plus qu'in élernaire infeins le mardit 


#### Generate text with the beam algorithm
- We need to loop for each character we want to generate, keeping track of the best ```beam_size``` sequences at the most.
- Besides keeping track of past generated character for each of these ```beam_size``` sequences, we need to keep track of their log-probability.
- This is done by, at each loop, keeping the ```beam_size```best predictions for each of the ```beam_size``` sequences, computing the log-probabilities of the newly formed (```beam_size```)$^2$ , and keeping the overall ```beam_size``` best new sequences.

In [89]:
def generate_beam(model, text, beam_size=16, num_generated=128):
    generated = text
    sentence = text[-maximum_seq_length:]
    # Initialization of the beam with log-probabilities for the sequence
    current_beam = [(0, [], sentence)]

    for l in range(num_generated):
        all_beams = []
        for prob, current_preds, current_input in current_beam:
            x = np.zeros((1, maximum_seq_length))
            for t, char in enumerate(current_input):
                x[0, t] = char_indices[char]
            prediction = model.predict(x = x, verbose = 0)[0]
            possible_next_chars = prediction.argsort()[-beam_size:][::-1]
            all_beams += [
                (prob + np.log(prediction[next_index]),
                 current_preds + [next_index],
                 current_input[1:] + indices_char[next_index]
                )
                for next_index in possible_next_chars]

        current_beam = sorted(all_beams)[-beam_size:]

    return text + ''.join([indices_char[idx] for idx in current_preds])

In [90]:
print(generate_beam(model_emb, text_ex))

La sottise, l'erreur, le péché dans les souvenirs.
 
 
 
 
   LE CHARE
 
 
   Je suis le soleil comme un soleil comme un grand comme un soleil comme un grand
